# Profiling iter_diag_spin with Scalene

This notebook uses Scalene to profile the iterative diagonalization function for spin chains.

In [1]:
# Load scalene extension
%load_ext scalene

# Import the function to profile
import sys
sys.path.insert(0, '..')
from bench.iter_diag import iter_diag_spin

Scalene extension successfully loaded. Note: Scalene currently only
supports CPU+GPU profiling inside Jupyter notebooks. For full Scalene
profiling, use the command line version. To profile in line mode, use
`%scrun [options] statement`. To profile in cell mode, use `%%scalene
[options]` followed by your code.

NOTE: in Jupyter notebook on MacOS, Scalene cannot profile child
processes. Do not run to try Scalene with multiprocessing in Jupyter
Notebook.


In [6]:
# Profile the function with scalene
# Run with moderate parameters to get meaningful profiling data
%scrun iter_diag_spin(N=100, Nkeep=2000, verbose=True)

[03:00:08] #01/100 : NK=1/2
[03:00:08] #02/100 : NK=2/4
[03:00:08] #03/100 : NK=4/8
[03:00:08] #04/100 : NK=8/16
[03:00:08] #05/100 : NK=16/32
[03:00:08] #06/100 : NK=32/64
[03:00:08] #07/100 : NK=64/128
[03:00:08] #08/100 : NK=128/256
[03:00:08] #09/100 : NK=256/512
[03:00:09] #10/100 : NK=512/1024
[03:00:10] #11/100 : NK=1024/2048
[03:00:15] #12/100 : NK=2000/4000
[03:00:19] #13/100 : NK=2000/4000
[03:00:23] #14/100 : NK=2000/4000
[03:00:27] #15/100 : NK=2000/4000
[03:00:31] #16/100 : NK=2000/4000
[03:00:35] #17/100 : NK=2000/4000
[03:00:39] #18/100 : NK=2000/4000
[03:00:45] #19/100 : NK=2000/4000
[03:00:50] #20/100 : NK=2000/4000
[03:00:56] #21/100 : NK=2000/4000
[03:01:02] #22/100 : NK=2000/4000
[03:01:07] #23/100 : NK=2000/4000
[03:01:13] #24/100 : NK=2000/4000
[03:01:19] #25/100 : NK=2000/4000
[03:01:24] #26/100 : NK=2000/4000
[03:01:30] #27/100 : NK=2000/4000
[03:01:36] #28/100 : NK=2000/4000
[03:01:42] #29/100 : NK=2000/4000
[03:01:47] #30/100 : NK=2000/4000
[03:01:53] #31/100 


Scalene: profile saved to scalene-profile.json
  To view in browser:  scalene view
  To view in terminal: scalene view --cli


## Profiling Setup

The `%scrun` magic command from Scalene will profile:
- CPU time usage
- Memory allocation
- GPU usage (if applicable)

Parameters used for profiling:
- `N=30`: Chain length (moderate size for meaningful timing)
- `Nkeep=200`: States to keep after truncation
- `verbose=False`: Suppress output for cleaner profiling

Run the cell above to see detailed profiling results.

In [5]:
_, _, _, AK_list = iter_diag_spin(N=100, Nkeep=2000, verbose=True)

[12:33:28] #01/100 : NK=1/2
[12:33:28] #02/100 : NK=2/4
[12:33:28] #03/100 : NK=4/8
[12:33:28] #04/100 : NK=8/16
[12:33:28] #05/100 : NK=16/32
[12:33:28] #06/100 : NK=32/64
[12:33:28] #07/100 : NK=64/128
[12:33:28] #08/100 : NK=128/256
[12:33:28] #09/100 : NK=256/512
[12:33:28] #10/100 : NK=512/1024
[12:33:29] #11/100 : NK=1024/2048
[12:33:33] #12/100 : NK=2000/4000
[12:33:38] #13/100 : NK=2000/4000
[12:33:41] #14/100 : NK=2000/4000
[12:33:45] #15/100 : NK=2000/4000
[12:33:49] #16/100 : NK=2000/4000
[12:33:53] #17/100 : NK=2000/4000
[12:33:57] #18/100 : NK=2000/4000
[12:34:03] #19/100 : NK=2000/4000
[12:34:08] #20/100 : NK=2000/4000
[12:34:14] #21/100 : NK=2000/4000
[12:34:19] #22/100 : NK=2000/4000
[12:34:25] #23/100 : NK=2000/4000
[12:34:30] #24/100 : NK=2000/4000
[12:34:36] #25/100 : NK=2000/4000
[12:34:42] #26/100 : NK=2000/4000
[12:34:47] #27/100 : NK=2000/4000
[12:34:53] #28/100 : NK=2000/4000
[12:34:59] #29/100 : NK=2000/4000
[12:35:04] #30/100 : NK=2000/4000
[12:35:10] #31/100 

In [7]:
%%scalene

from nicole import subsector, decomp, contract

# Convert left-canonical MPS to right-canonical
# Create a copy to avoid overwriting the original
AK_right = AK_list.copy()

# Sweep from right to left
for i in range(len(AK_right) - 1, 0, -1):
    # Decompose: split left index from (left, right, phys)
    # axes=[0] means we decompose along axis 0 (left index)
    # LV mode with << flow: L (with S) goes left, V (isometry) stays here
    # After decomp: L has shape (left, left'), V has shape (left', right, phys)
    L, V = decomp(AK_right[i], axes=0, flow='<<', mode='LV', trunc={'nkeep': 2000})
    AK_right[i] = V  # Isometry stays at current position
    AK_right[i].retag(0, f'R{i-1:02d}')
    # Contract L into left neighbor: AK[i-1](left, right, phys) * L(left, left')
    # Contracting axis 1 (right) of AK[i-1] with axis 0 (left) of L
    AK_right[i-1] = contract(AK_right[i-1], L, axes=(1, 0), perm=(0, 2, 1))
    AK_right[i-1].retag([0, 1], [f'R{i-2:02d}', f'R{i-1:02d}'])

print(f"Converted to right-canonical form with nkeep=2000")
print(f"Final tensors in AK_right: {len(AK_right)}")

Converted to right-canonical form with nkeep=2000
Final tensors in AK_right: 100



Scalene: profile saved to scalene-profile.json
  To view in browser:  scalene view
  To view in terminal: scalene view --cli


In [9]:
AK_right[50]


  info:  3x { 16 x 1 }  having 'A'    Tensor,  { R49*, R50, s50 }
  data:  3-D complex128 (25.3 MB)    2000 x 2000 x 2 => 2000 x 2000 x 2  @ norm = 44.7214

     1.  457x559x1 |  1x1x1   [ -2 ; -1 ; -1 ]  3.9 MB
     2.  457x326x1 |  1x1x1   [ -2 ; -3 ;  1 ] 2.27 MB
     3.  202x326x1 |  1x1x1   [ -4 ; -3 ; -1 ]    1 MB
     4.  202x102x1 |  1x1x1   [ -4 ; -5 ;  1 ]  322 kB
     5.  42x102x1 |  1x1x1   [ -6 ; -5 ; -1 ] 66.9 kB
     6.  42x12x1 |  1x1x1   [ -6 ; -7 ;  1 ] 7.88 kB
     7.  2x12x1  |  1x1x1   [ -8 ; -7 ; -1 ]   384 B
     8.  593x559x1 |  1x1x1   [  0 ; -1 ;  1 ] 5.06 MB
     9.  593x560x1 |  1x1x1   [  0 ;  1 ; -1 ] 5.07 MB
    ... (7 more)